In [1]:
import os
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [ ]:
# train ->List[file_name]
# x_max > orign w :  [276, 561, 566, 16619, 17988, 26798]
# y_max > orign h :  [22706, 26573]
# x_min < 0:  [5177, 6733, 12059, 13632, 20671, 21533]
# y_min < 0:  [5225, 10205, 10205]

# val ->List[file_name]
# x_max > orign w :  [3502, 3503, 3504, 3505, 3506, 8360, 23155]
# y_max > orign h :  [4767, 4767, 4768, 4769, 4770, 4770, 4774, 4774, 4789]
# x_min < 0:  [25320]
# y_min < 0:  [23082]

In [2]:
root_path = '/workspace/traffic_light/data/detection/val'
coco = COCO(os.path.join(root_path, 'val.json'))

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [89]:
def modify_annos(root_path, anno_id, box):
    base = os.path.basename(root_path)
    with open(os.path.join(root_path, f'{base}.json'), 'r') as f:
        coco_data = json.load(f)
        for annotation in coco_data['annotations']:
            if annotation['id'] == anno_id:
                annotation['bbox'] = box

    with open(os.path.join(root_path, f'{base}.json'), 'w') as f:
        json.dump(coco_data, f, indent=4)

cnt = 0
for _id in tqdm(sorted(coco.getImgIds())):
    # _id = 276
    # print(coco.loadImgs(_id))
    w = coco.loadImgs(_id)[0]['width']
    h = coco.loadImgs(_id)[0]['height']
    # print(coco.getAnnIds(_id))
    for i in coco.getAnnIds(_id):
        need_modify = False
        bbox = coco.loadAnns(i)[0]['bbox']
        x_max = bbox[0]+bbox[2]
        x_min = bbox[0]
        y_max = bbox[1]+bbox[3]
        y_min = bbox[1]
        if x_max > w:
            print()
            print('x_max', coco.loadAnns(i))
            bbox[2] = float(w-bbox[0])
            need_modify = True
            cnt +=1

        if x_min < 0:
            print(f'(w:{w},h:{h}')
            print('x_min',coco.loadAnns(i))
            bbox[0] = float(0)
            need_modify = True
            cnt +=1

        if y_max > h:
            print()
            print('y_max',coco.loadAnns(i))
            bbox[3] = float(h-bbox[1])
            need_modify = True
            cnt +=1

        if y_min < 0:
            print(f'(w:{w},h:{h}')
            print('y_min', coco.loadAnns(i))
            bbox[1] = float(0)
            need_modify = True
            cnt +=1

        
        # if need_modify:
        #     modify_annos(root_path, i, bbox)
print('cnt:',cnt)

100%|██████████| 2686/2686 [00:00<00:00, 207185.17it/s]

cnt: 0


In [8]:
import pickle

# 파일 열기
with open('/workspace/traffic_light/mmdetection/outputs_result/v1.pkl', 'rb') as file:
    data = pickle.load(file)
    # print(len(data[0].keys())
    print(data[0].keys())
    # print(data[0]['pad_shape'])
    print(data[0]['pred_instances'].keys())
    print(len(data[0]['pred_instances']['scores']))
    print(len(data[0]['pred_instances']['bboxes']))
    print(data[0]['pred_instances']['bboxes'])
    print(data[0]['img_id'])
    print(data[0]['img_path'])
    print(data[0]['img_shape'])
    print(data[0]['ori_shape'])
    
# print(data)

dict_keys(['ori_shape', 'img_path', 'pad_shape', 'img_id', 'img_shape', 'batch_input_shape', 'scale_factor', 'pred_instances'])
dict_keys(['bboxes', 'labels', 'scores'])
300
300
tensor([[507.0966, 344.4064, 548.6027, 354.9404],
        [330.9306, 348.4921, 371.2928, 359.2637],
        [472.9115, 427.7584, 490.9619, 434.1396],
        ...,
        [201.8082, 436.5136, 215.3690, 444.6447],
        [415.6876, 456.8303, 424.1629, 462.9742],
        [691.5817, 465.6791, 704.4044, 473.9737]])
10000000
/workspace/traffic_light/data/detection/test/images/10000000.jpg
(800, 1333)
(768, 1280)
